In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import sys
import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3
import requests
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_44d4310629d4414889cf904a19adba4c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='UYdfinbjczjz-WXwtEzkxnc3lnPlW9rkJRJdqDgAQWhH',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_44d4310629d4414889cf904a19adba4c.get_object(Bucket='course-donotdelete-pr-phhvmbfasslhdb',Key='beijingstation2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1



In [61]:
df_data_1

,Station,Latitude,Longitude,Cluster Labels
0,XIERQI,40.052243,116.306144,2
1,SHANGDI,40.032958,116.320519,4
2,WUDAOKOU,39.992833,116.337780,2
3,ZHICHUNLU,39.976424,116.340141,1
4,DAZHONGSI,39.966923,116.345126,4
5,XIZHIMEN,39.941856,116.353234,4
6,CHEGONGZHUANG,39.931445,116.356180,1
7,DONGZHIMEN,39.941255,116.433859,2
8,HUIXINXIJIENANKOU,39.976998,116.417644,2
9,SANYUANQIAO,39.960879,116.457055,4


In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed and imported!


In [3]:
latitude = 39.992833
longitude = 116.337780
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=10)


In [4]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, station in zip(df_data_1.Latitude, df_data_1.Longitude,df_data_1.Station):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

In [5]:
CLIENT_ID = 'QQL1NTDTJMZWQU5MG4T5HGNH51Z1YHZ4AJIBMIUQ44ZOJL5C' # your Foursquare ID
CLIENT_SECRET = 'H5UOOLKASRDBYN5PH2ITL0OSO4FR0RDOGKLJCE31ZJCJ0FT5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QQL1NTDTJMZWQU5MG4T5HGNH51Z1YHZ4AJIBMIUQ44ZOJL5C
CLIENT_SECRET:H5UOOLKASRDBYN5PH2ITL0OSO4FR0RDOGKLJCE31ZJCJ0FT5


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
cz_venues = getNearbyVenues(names=df_data_1['Station'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )


XIERQI
SHANGDI
WUDAOKOU
ZHICHUNLU
DAZHONGSI
XIZHIMEN
CHEGONGZHUANG
DONGZHIMEN
HUIXINXIJIENANKOU
SANYUANQIAO
TUANJIEHU
CHAOYANGMEN
HAIDIANHUANGZHUANG
AOTIZHONGXIN
WANGJINGNAN
DONGSISHITIAO
XIDAN
SHUANGJING
QINGNIANLU
WEIGONGCUN


In [10]:
print(cz_venues.shape)
cz_venues

(1468, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.046375,116.292217,Coffee Shop
1,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.052782,116.296437,Coffee Shop
2,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.034749,116.302546,Coffee Shop
3,XIERQI,40.052243,116.306144,Starbucks (星巴克),40.041113,116.303783,Coffee Shop
4,XIERQI,40.052243,116.306144,潇湘府,40.042878,116.295512,Hunan Restaurant
5,XIERQI,40.052243,116.306144,Holiday Inn Express Shangdi Beijing 北京上地智选假日酒店,40.035402,116.308608,Hotel
6,XIERQI,40.052243,116.306144,Taishan Hotel (泰山饭店),40.048264,116.323578,Hotel
7,XIERQI,40.052243,116.306144,McDonald's (McDonald's (麦当劳)),40.051646,116.296647,Fast Food Restaurant
8,XIERQI,40.052243,116.306144,西二旗益民市场,40.051226,116.307292,Farmers Market
9,XIERQI,40.052243,116.306144,Yitel 和颐酒店,40.043265,116.290185,Hotel


In [11]:
cz_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AOTIZHONGXIN,63,63,63,63,63,63
CHAOYANGMEN,100,100,100,100,100,100
CHEGONGZHUANG,67,67,67,67,67,67
DAZHONGSI,55,55,55,55,55,55
DONGSISHITIAO,100,100,100,100,100,100
DONGZHIMEN,100,100,100,100,100,100
HAIDIANHUANGZHUANG,100,100,100,100,100,100
HUIXINXIJIENANKOU,57,57,57,57,57,57
QINGNIANLU,40,40,40,40,40,40


In [12]:
print('There are {} uniques categories.'.format(len(cz_venues['Venue Category'].unique())))

There are 155 uniques categories.


In [13]:
# one hot encoding
cz_onehot = pd.get_dummies(cz_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cz_onehot['Neighborhood'] = cz_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cz_onehot.columns[-1]] + list(cz_onehot.columns[:-1])
cz_onehot = cz_onehot[fixed_columns]

cz_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo,Zoo Exhibit
0,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,XIERQI,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
cz_onehot.shape

(1468, 156)

In [15]:
cz_grouped = cz_onehot.groupby('Neighborhood').mean().reset_index()
cz_grouped

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant,Zoo,Zoo Exhibit
0,AOTIZHONGXIN,0.000000,0.00,0.000000,0.00,0.015873,0.00,0.00,0.047619,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
1,CHAOYANGMEN,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.010000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.00,0.010000,0.00,0.000000,0.000000
2,CHEGONGZHUANG,0.014925,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.014925,0.00,0.000000,0.00,0.000000,0.000000
3,DAZHONGSI,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.018182,...,0.000000,0.00,0.00,0.00,0.018182,0.00,0.000000,0.00,0.000000,0.000000
4,DONGSISHITIAO,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.010000,0.000000,...,0.000000,0.01,0.00,0.00,0.010000,0.01,0.010000,0.00,0.000000,0.000000
5,DONGZHIMEN,0.000000,0.00,0.000000,0.01,0.000000,0.00,0.00,0.010000,0.000000,...,0.010000,0.01,0.01,0.00,0.010000,0.01,0.020000,0.00,0.000000,0.000000
6,HAIDIANHUANGZHUANG,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.010000,0.020000,...,0.010000,0.00,0.00,0.01,0.000000,0.00,0.020000,0.00,0.000000,0.000000
7,HUIXINXIJIENANKOU,0.017544,0.00,0.000000,0.00,0.000000,0.00,0.00,0.070175,0.017544,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
8,QINGNIANLU,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000
9,SANYUANQIAO,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.020000,0.020000,...,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000


In [16]:
cz_grouped.shape

(20, 156)

In [17]:
num_top_venues = 5

for hood in cz_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cz_grouped[cz_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AOTIZHONGXIN----
                  venue  freq
0    Chinese Restaurant  0.14
1                 Hotel  0.13
2           Coffee Shop  0.11
3  Fast Food Restaurant  0.08
4      Asian Restaurant  0.05


----CHAOYANGMEN----
                venue  freq
0               Hotel  0.11
1  Italian Restaurant  0.05
2       Shopping Mall  0.05
3        Cocktail Bar  0.05
4  Chinese Restaurant  0.04


----CHEGONGZHUANG----
                  venue  freq
0           Coffee Shop  0.10
1                 Hotel  0.10
2    Chinese Restaurant  0.07
3      Department Store  0.06
4  Fast Food Restaurant  0.06


----DAZHONGSI----
                  venue  freq
0  Fast Food Restaurant  0.18
1    Chinese Restaurant  0.16
2                  Café  0.07
3           Coffee Shop  0.05
4         Movie Theater  0.04


----DONGSISHITIAO----
                    venue  freq
0                   Hotel  0.08
1     Japanese Restaurant  0.05
2           Shopping Mall  0.05
3  Peking Duck Restaurant  0.04
4            Cocktail

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cz_grouped['Neighborhood']

for ind in np.arange(cz_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cz_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AOTIZHONGXIN,Chinese Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Asian Restaurant,Park,Stadium,Lounge,Shopping Mall,Bus Stop
1,CHAOYANGMEN,Hotel,Shopping Mall,Italian Restaurant,Cocktail Bar,Chinese Restaurant,Café,Japanese Restaurant,Coffee Shop,Eastern European Restaurant,Peking Duck Restaurant
2,CHEGONGZHUANG,Coffee Shop,Hotel,Chinese Restaurant,Department Store,Fast Food Restaurant,Café,Theater,Hunan Restaurant,Shopping Mall,Hostel
3,DAZHONGSI,Fast Food Restaurant,Chinese Restaurant,Café,Coffee Shop,Restaurant,Shopping Mall,Convenience Store,Grocery Store,Movie Theater,Hotel
4,DONGSISHITIAO,Hotel,Shopping Mall,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Peking Duck Restaurant,Szechuan Restaurant,Café,Brewery,Coffee Shop
5,DONGZHIMEN,Hotel,Japanese Restaurant,Brewery,Chinese Restaurant,Peking Duck Restaurant,Café,Cocktail Bar,Shopping Mall,Coffee Shop,Spanish Restaurant
6,HAIDIANHUANGZHUANG,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Café,Hotel,Bar,Department Store,Bakery,Multiplex,Clothing Store
7,HUIXINXIJIENANKOU,Coffee Shop,Chinese Restaurant,Café,Asian Restaurant,Hotel,Gym,Pizza Place,Multiplex,Fast Food Restaurant,Karaoke Bar
8,QINGNIANLU,Chinese Restaurant,Supermarket,Coffee Shop,Hotpot Restaurant,Hotel,Clothing Store,Shopping Mall,Multiplex,Italian Restaurant,Skating Rink
9,SANYUANQIAO,Japanese Restaurant,Hotel,Chinese Restaurant,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Mexican Restaurant,New American Restaurant


In [24]:
kclusters = 5

cz_grouped_clustering = cz_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cz_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 2, 1, 4, 4, 1, 2, 2, 4], dtype=int32)

In [28]:
cz_merged = df_data_1

# add clustering labels
cz_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cz_merged = cz_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Station')

cz_merged # check the last columns!

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,XIERQI,40.052243,116.306144,2,Coffee Shop,Hotel,Fast Food Restaurant,Hunan Restaurant,Korean Restaurant,Farmers Market,Chinese Restaurant,Food Court,Asian Restaurant,Café
1,SHANGDI,40.032958,116.320519,4,Coffee Shop,Chinese Restaurant,Hotel,Fast Food Restaurant,Cantonese Restaurant,Bus Stop,Tea Room,Park,Department Store,Shopping Mall
2,WUDAOKOU,39.992833,116.337780,2,Café,Chinese Restaurant,Fast Food Restaurant,Korean Restaurant,Coffee Shop,Bar,Xinjiang Restaurant,Hotel,Japanese Restaurant,Bakery
3,ZHICHUNLU,39.976424,116.340141,1,Chinese Restaurant,Fast Food Restaurant,Café,Korean Restaurant,Hotel,Coffee Shop,Bar,Bakery,Szechuan Restaurant,Dessert Shop
4,DAZHONGSI,39.966923,116.345126,4,Fast Food Restaurant,Chinese Restaurant,Café,Coffee Shop,Restaurant,Shopping Mall,Convenience Store,Grocery Store,Movie Theater,Hotel
5,XIZHIMEN,39.941856,116.353234,4,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Café,Shopping Mall,Zoo Exhibit,Stadium,Russian Restaurant,Hotpot Restaurant,Planetarium
6,CHEGONGZHUANG,39.931445,116.356180,1,Coffee Shop,Hotel,Chinese Restaurant,Department Store,Fast Food Restaurant,Café,Theater,Hunan Restaurant,Shopping Mall,Hostel
7,DONGZHIMEN,39.941255,116.433859,2,Hotel,Japanese Restaurant,Brewery,Chinese Restaurant,Peking Duck Restaurant,Café,Cocktail Bar,Shopping Mall,Coffee Shop,Spanish Restaurant
8,HUIXINXIJIENANKOU,39.976998,116.417644,2,Coffee Shop,Chinese Restaurant,Café,Asian Restaurant,Hotel,Gym,Pizza Place,Multiplex,Fast Food Restaurant,Karaoke Bar
9,SANYUANQIAO,39.960879,116.457055,4,Japanese Restaurant,Hotel,Chinese Restaurant,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Mexican Restaurant,New American Restaurant


In [27]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cz_merged['Latitude'], cz_merged['Longitude'], cz_merged['Station'], cz_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
cz_merged

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,XIERQI,40.052243,116.306144,2,Coffee Shop,Hotel,Fast Food Restaurant,Hunan Restaurant,Korean Restaurant,Farmers Market,Chinese Restaurant,Food Court,Asian Restaurant,Café
1,SHANGDI,40.032958,116.320519,4,Coffee Shop,Chinese Restaurant,Hotel,Fast Food Restaurant,Cantonese Restaurant,Bus Stop,Tea Room,Park,Department Store,Shopping Mall
2,WUDAOKOU,39.992833,116.337780,2,Café,Chinese Restaurant,Fast Food Restaurant,Korean Restaurant,Coffee Shop,Bar,Xinjiang Restaurant,Hotel,Japanese Restaurant,Bakery
3,ZHICHUNLU,39.976424,116.340141,1,Chinese Restaurant,Fast Food Restaurant,Café,Korean Restaurant,Hotel,Coffee Shop,Bar,Bakery,Szechuan Restaurant,Dessert Shop
4,DAZHONGSI,39.966923,116.345126,4,Fast Food Restaurant,Chinese Restaurant,Café,Coffee Shop,Restaurant,Shopping Mall,Convenience Store,Grocery Store,Movie Theater,Hotel
5,XIZHIMEN,39.941856,116.353234,4,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Café,Shopping Mall,Zoo Exhibit,Stadium,Russian Restaurant,Hotpot Restaurant,Planetarium
6,CHEGONGZHUANG,39.931445,116.356180,1,Coffee Shop,Hotel,Chinese Restaurant,Department Store,Fast Food Restaurant,Café,Theater,Hunan Restaurant,Shopping Mall,Hostel
7,DONGZHIMEN,39.941255,116.433859,2,Hotel,Japanese Restaurant,Brewery,Chinese Restaurant,Peking Duck Restaurant,Café,Cocktail Bar,Shopping Mall,Coffee Shop,Spanish Restaurant
8,HUIXINXIJIENANKOU,39.976998,116.417644,2,Coffee Shop,Chinese Restaurant,Café,Asian Restaurant,Hotel,Gym,Pizza Place,Multiplex,Fast Food Restaurant,Karaoke Bar
9,SANYUANQIAO,39.960879,116.457055,4,Japanese Restaurant,Hotel,Chinese Restaurant,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Mexican Restaurant,New American Restaurant


In [38]:
test=cz_merged.loc[cz_merged['Cluster Labels'] == 0]
test

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,SHUANGJING,39.893557,116.461962,0,Coffee Shop,Chinese Restaurant,Hotel,Café,Fast Food Restaurant,Shopping Mall,Multiplex,Supermarket,Hotel Bar,Italian Restaurant


In [51]:
test2=cz_merged.loc[cz_merged['Cluster Labels'] == 1]
test2

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,ZHICHUNLU,39.976424,116.340141,1,Chinese Restaurant,Fast Food Restaurant,Café,Korean Restaurant,Hotel,Coffee Shop,Bar,Bakery,Szechuan Restaurant,Dessert Shop
6,CHEGONGZHUANG,39.931445,116.356180,1,Coffee Shop,Hotel,Chinese Restaurant,Department Store,Fast Food Restaurant,Café,Theater,Hunan Restaurant,Shopping Mall,Hostel
13,AOTIZHONGXIN,39.986407,116.394155,1,Chinese Restaurant,Hotel,Coffee Shop,Fast Food Restaurant,Asian Restaurant,Park,Stadium,Lounge,Shopping Mall,Bus Stop
15,DONGSISHITIAO,39.933852,116.434333,1,Hotel,Shopping Mall,Japanese Restaurant,Cocktail Bar,Chinese Restaurant,Peking Duck Restaurant,Szechuan Restaurant,Café,Brewery,Coffee Shop
18,QINGNIANLU,39.923018,116.517454,1,Chinese Restaurant,Supermarket,Coffee Shop,Hotpot Restaurant,Hotel,Clothing Store,Shopping Mall,Multiplex,Italian Restaurant,Skating Rink
19,WEIGONGCUN,39.957765,116.323136,1,Hotel,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Café,Pizza Place,Movie Theater,Department Store,Grocery Store,Museum


In [53]:
test3=cz_merged.loc[cz_merged['Cluster Labels'] == 2]
test3

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,XIERQI,40.052243,116.306144,2,Coffee Shop,Hotel,Fast Food Restaurant,Hunan Restaurant,Korean Restaurant,Farmers Market,Chinese Restaurant,Food Court,Asian Restaurant,Café
2,WUDAOKOU,39.992833,116.337780,2,Café,Chinese Restaurant,Fast Food Restaurant,Korean Restaurant,Coffee Shop,Bar,Xinjiang Restaurant,Hotel,Japanese Restaurant,Bakery
7,DONGZHIMEN,39.941255,116.433859,2,Hotel,Japanese Restaurant,Brewery,Chinese Restaurant,Peking Duck Restaurant,Café,Cocktail Bar,Shopping Mall,Coffee Shop,Spanish Restaurant
8,HUIXINXIJIENANKOU,39.976998,116.417644,2,Coffee Shop,Chinese Restaurant,Café,Asian Restaurant,Hotel,Gym,Pizza Place,Multiplex,Fast Food Restaurant,Karaoke Bar
10,TUANJIEHU,39.933450,116.461705,2,Japanese Restaurant,Hotel,Cocktail Bar,Chinese Restaurant,Café,Brewery,French Restaurant,Shopping Mall,Italian Restaurant,Peking Duck Restaurant
11,CHAOYANGMEN,39.924540,116.433433,2,Hotel,Shopping Mall,Italian Restaurant,Cocktail Bar,Chinese Restaurant,Café,Japanese Restaurant,Coffee Shop,Eastern European Restaurant,Peking Duck Restaurant
14,WANGJINGNAN,39.984704,116.482311,2,Café,Coffee Shop,Chinese Restaurant,Korean Restaurant,Hotel,Art Gallery,Shopping Mall,American Restaurant,Japanese Restaurant,Cantonese Restaurant


In [41]:
cz_merged.loc[cz_merged['Cluster Labels'] == 3]

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,XIDAN,39.907769,116.374751,3,Historic Site,Coffee Shop,Hotel,Chinese Restaurant,Shopping Mall,Department Store,Café,Hostel,Bookstore,History Museum


In [55]:
test5=cz_merged.loc[cz_merged['Cluster Labels'] == 4]
test5

,Station,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,SHANGDI,40.032958,116.320519,4,Coffee Shop,Chinese Restaurant,Hotel,Fast Food Restaurant,Cantonese Restaurant,Bus Stop,Tea Room,Park,Department Store,Shopping Mall
4,DAZHONGSI,39.966923,116.345126,4,Fast Food Restaurant,Chinese Restaurant,Café,Coffee Shop,Restaurant,Shopping Mall,Convenience Store,Grocery Store,Movie Theater,Hotel
5,XIZHIMEN,39.941856,116.353234,4,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Café,Shopping Mall,Zoo Exhibit,Stadium,Russian Restaurant,Hotpot Restaurant,Planetarium
9,SANYUANQIAO,39.960879,116.457055,4,Japanese Restaurant,Hotel,Chinese Restaurant,Coffee Shop,Café,Italian Restaurant,Park,Bakery,Mexican Restaurant,New American Restaurant
12,HAIDIANHUANGZHUANG,39.976909,116.317043,4,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Café,Hotel,Bar,Department Store,Bakery,Multiplex,Clothing Store


In [58]:
#test['1st Most Common Venue']
#df3 = pd.DataFrame({'ca':test['1st Most Common Venue']+test['2st Most Common Venue']})
#df3
#列拼接合并，合一
#del test2['Station']
#del test2['Latitude']
#del test2['Longitude']
#del test2['Cluster Labels']
df2=pd.concat(test2.iloc[:,i] for i in range(test2.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
df2.value_counts() 
#df2.unique()

Coffee Shop               6
Chinese Restaurant        6
Hotel                     6
Shopping Mall             4
Fast Food Restaurant      4
Café                      4
Department Store          2
Szechuan Restaurant       2
Cocktail Bar              1
Pizza Place               1
Multiplex                 1
Italian Restaurant        1
Korean Restaurant         1
Supermarket               1
Movie Theater             1
Asian Restaurant          1
Hunan Restaurant          1
Theater                   1
Lounge                    1
Clothing Store            1
Bar                       1
Hostel                    1
Park                      1
Brewery                   1
Dessert Shop              1
Bus Stop                  1
Skating Rink              1
Grocery Store             1
Peking Duck Restaurant    1
Stadium                   1
Museum                    1
Hotpot Restaurant         1
Japanese Restaurant       1
Bakery                    1
dtype: int64

In [56]:
del test5['Station']
del test5['Latitude']
del test5['Longitude']
del test5['Cluster Labels']
df2=pd.concat(test.iloc[:,i] for i in range(test5.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
df2.value_counts() 

Chinese Restaurant         5
Coffee Shop                5
Fast Food Restaurant       4
Hotel                      4
Café                       4
Shopping Mall              3
Bakery                     2
Park                       2
Department Store           2
Bus Stop                   1
Italian Restaurant         1
Cantonese Restaurant       1
Clothing Store             1
Convenience Store          1
Planetarium                1
Zoo Exhibit                1
Russian Restaurant         1
Stadium                    1
Restaurant                 1
Mexican Restaurant         1
Japanese Restaurant        1
Hotpot Restaurant          1
Bar                        1
New American Restaurant    1
Tea Room                   1
Grocery Store              1
Movie Theater              1
Multiplex                  1
dtype: int64

In [60]:
#del test3['Station']
#del test3['Latitude']
#del test3['Longitude']
#del test3['Cluster Labels']
df2=pd.concat(test3.iloc[:,i] for i in range(test3.shape[1]))
#适当修改索引
df2.index=np.arange(len(df2))
df2.value_counts() 

Hotel                          7
Chinese Restaurant             7
Café                           7
Coffee Shop                    6
Japanese Restaurant            5
Shopping Mall                  4
Korean Restaurant              3
Cocktail Bar                   3
Fast Food Restaurant           3
Peking Duck Restaurant         3
Italian Restaurant             2
Brewery                        2
Asian Restaurant               2
French Restaurant              1
Gym                            1
American Restaurant            1
Spanish Restaurant             1
Multiplex                      1
Eastern European Restaurant    1
Cantonese Restaurant           1
Art Gallery                    1
Xinjiang Restaurant            1
Karaoke Bar                    1
Hunan Restaurant               1
Farmers Market                 1
Food Court                     1
Bar                            1
Pizza Place                    1
Bakery                         1
dtype: int64